<a href="https://colab.research.google.com/github/Kushagra2000/Neural_machine_translation/blob/main/Machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#imports
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
import pandas as pd
import numpy as np
import re
import unicodedata
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Activation,InputLayer,Embedding
from tensorflow.keras.losses import sparse_categorical_crossentropy,categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
# !pip install -q -U keras-tuner
# import kerastuner as kt 
import IPython
from keras.utils import to_categorical
tf.test.gpu_device_name()

TensorFlow 1.x selected.
1.15.2


Using TensorFlow backend.


'/device:GPU:0'

In [ ]:
#Upload the deu.txt file and put it in the Data Folder
#!ls Data/

In [ ]:
#reading lines from the txt file
num_examples=20000
with open('drive/MyDrive/deu.txt','r',encoding='utf-8') as f:
  lines=(f.readlines())
lines=lines[:num_examples]

In [ ]:
#Hyperparameters go here

test_size=0.3
m1_lr=0.001
m2_lr=0.005
m1_lstm_units=64
m2_lstm_units=64
m1_epochs=25
m2_epochs=30
m1_batch_size=128
m2_batch_size=128
m2_embedding_col=64

In [ ]:
#removing the unnecessary data and obtaining sentence pairs 
def process_text(lines,translate_mode):
  ''' 
  Takes a list of tab seperated
  English and German sentences as input 
  and returns dataframe of processed
  English and German sentence pairs
  '''
  proc_lines=[]
  if translate_mode==False:
    for line in lines:
      line=line.strip() #to remove newlines at the end of a sentence
      line=line.split("\t") #splitting by tabs
      line=line[:-1]  #remove contributing information
      
      line[0]=''.join(c for c in unicodedata.normalize('NFD', line[0]) if unicodedata.category(c) != 'Mn')
      line[0]=line[0].encode('utf8','ignore').decode('utf8')
      line[0]=line[0].replace(u'\u200b',' ')
      line[0]=line[0].lower()
      line[0]=line[0].replace('\xa0', ' ')
      line[0]=re.sub(r"([.,?!;])",r" \1 ",line[0])   #adding spaces before and after punctuation
      line[0]=re.sub(r"[0-9]"," ",line[0])  #removing numbers
      line[0]=re.sub(r'["]'," ",line[0])  #removing quotes
      line[0]=re.sub(r"[']","",line[0])
      line[0]=re.sub(r"[%-,]"," ",line[0])
      line[0]=re.sub(r"[:]"," ",line[0])
      line[0]=re.sub(r'[" "]+'," ",line[0])  #removing excess spaces
      line[0]=line[0].strip() #removing spaces from the end of string
      line[0]="<SOS> "+line[0]+" <EOS>"

      line[1]=''.join(c for c in unicodedata.normalize('NFD', line[1]) if unicodedata.category(c) != 'Mn')      
      line[1]=line[1].replace(u'\u200b',' ')
      line[1]=line[1].replace('\xa0', ' ')
      line[1]=line[1].lower()
      line[1]=re.sub(r"([.,?!;])",r" \1 ",line[1])
      line[1]=re.sub(r"[0-9]",r" ",line[1])
      line[1]=re.sub(r'["]',"",line[1])
      line[1]=re.sub(r'[—]',"",line[1])
      line[1]=re.sub(r'[„]',"",line[1])
      line[1]=re.sub(r'[“]',"",line[1])
      line[1]=re.sub(r'[–]',"",line[1])
      line[1]=re.sub(r'[‘‚]',"",line[1])
      line[1]=re.sub(r"[']","",line[1])
      line[1]=re.sub(r"[%()-,]"," ",line[1])
      line[1]=re.sub(r"[:]"," ",line[1])
      line[1]=re.sub(r'[" "]+'," ",line[1])
      line[1]=line[1].strip()
      line[1]="<SOS> "+line[1]+" <EOS>"

      proc_lines.append(line) 
    return pd.DataFrame.from_records(data=proc_lines,columns=['eng','ger'])
  else:
    lines=lines.strip()
    lines=''.join(c for c in unicodedata.normalize('NFD', lines) if unicodedata.category(c) != 'Mn')
    lines=lines.encode('utf8','ignore').decode('utf8')
    lines=lines.replace(u'\u200b',' ')
    lines=lines.lower()
    lines=lines.replace('\xa0', ' ')
    lines=re.sub(r"([.,?!;])",r" \1 ",lines)   #adding spaces before and after punctuation
    lines=re.sub(r"[0-9]"," ",lines)
    lines=re.sub(r'["]'," ",lines)
    lines=re.sub(r"[']","",lines)
    lines=re.sub(r"[%-,]"," ",lines)
    lines=re.sub(r"[:]"," ",lines)
    lines=re.sub(r'[" "]+'," ",lines)  #removing excess spaces
    lines=lines.strip() #removing spaces from the end of string
    lines="<SOS> "+lines+" <EOS>"
    return lines

  
     
  

In [ ]:
#reading in the dataset, sampling randomly, shuffling and removing duplicates 
df=process_text(lines,False)
#df=df.sample(n=num_examples, random_state=1)
df.drop_duplicates(subset="eng",inplace=True)
df = df.sample(frac=1).reset_index(drop=True)
df.head(10)

,eng,ger
0,<SOS> youre decisive . <EOS>,<SOS> du bist entschlossen . <EOS>
1,<SOS> can i hug you ? <EOS>,<SOS> darf ich dich umarmen ? <EOS>
2,<SOS> im a good shot . <EOS>,<SOS> ich bin ein guter schutze . <EOS>
3,<SOS> what wasnt easy ? <EOS>,<SOS> was war nicht einfach ? <EOS>
4,<SOS> i must be blind . <EOS>,<SOS> ich muss blind sein . <EOS>
5,<SOS> was tom here ? <EOS>,<SOS> war tom hier ? <EOS>
6,<SOS> hes your friend . <EOS>,<SOS> er ist dein freund . <EOS>
7,<SOS> dont be a baby . <EOS>,<SOS> benimm dich nicht wie ein baby ! <EOS>
8,<SOS> why do you lie ? <EOS>,<SOS> wieso lugst du ? <EOS>
9,<SOS> i wanted to win . <EOS>,<SOS> ich wollte gewinnen . <EOS>


In [ ]:
df.tail(10)

,eng,ger
14840,<SOS> ive got money . <EOS>,<SOS> ich habe geld . <EOS>
14841,<SOS> they liked tom . <EOS>,<SOS> sie mochten tom . <EOS>
14842,<SOS> im here again . <EOS>,<SOS> ich bin wieder hier . <EOS>
14843,<SOS> help me fix this . <EOS>,<SOS> hilf mir das zu reparieren . <EOS>
14844,<SOS> dont rub it in . <EOS>,<SOS> streu nicht noch salz in die wunde ! <EOS>
14845,<SOS> this is a sign . <EOS>,<SOS> das ist ein zeichen ! <EOS>
14846,<SOS> tom is devoted . <EOS>,<SOS> tom ist hingebungsvoll . <EOS>
14847,<SOS> its early . <EOS>,<SOS> es ist fruh . <EOS>
14848,<SOS> we went dancing . <EOS>,<SOS> wir gingen tanzen . <EOS>
14849,<SOS> that is his car . <EOS>,<SOS> das ist sein auto . <EOS>


In [ ]:
class Vocab_builder():
  '''
  Builds vocabulary and 
  word to index and index to word dictionaries
  from dataset
  '''
  def __init__(self,lang,series):
    self.lang=lang
    self.data=series
  def tokenize(self,line):
    return line.split(' ')
  def build_vocab(self):
    self.uniq_words=set()
    
    self.maxlen=0
    count=3
    self.num_list=[]
    for index,line in self.data.items():
      self.word_list=self.tokenize(line)
      self.maxlen=max(len(self.word_list),self.maxlen)
      for word in self.word_list:
        if(word not in self.uniq_words and word!='<EOS>' and word!='<SOS>'):
          self.uniq_words.add(word)
          self.num_list.append(count)
          count+=1
      
    self.vocab_list=['<PAD>','<SOS>','<EOS>']+sorted(list(self.uniq_words))
    self.num_list=[0,1,2]+self.num_list
    print("Built vocabulary having {} elements".format(len(self.vocab_list)))
    print("Largest sentence length (with tags):{}".format(self.maxlen))
    return dict(zip(self.vocab_list,self.num_list)),dict(zip(self.num_list,self.vocab_list))






In [ ]:
#Objects of Vocab_builder class
eng=Vocab_builder('eng',df['eng'])
ger=Vocab_builder('ger',df['ger'])


In [ ]:
#English word to index and index to word dictionaries
eng_w2i,eng_i2w=eng.build_vocab()


Built vocabulary having 3655 elements
Largest sentence length (with tags):9


In [ ]:
#checking for special characters in English dictionary
for i in sorted (eng_w2i.keys())[:20] :  
     print(i, end = " ") 

! $ - . <EOS> <PAD> <SOS> ? a aah abandon abandoned abated abhor able aboard about abroad absent absurd 

In [ ]:
#German word to index and index to word dictionaries
ger_w2i,ger_i2w=ger.build_vocab()

Built vocabulary having 4892 elements
Largest sentence length (with tags):13


In [ ]:
#checking if the index dictionaries are correct
'geh'==ger_i2w[ger_w2i['geh']]

True

In [ ]:
'go'==eng_i2w[eng_w2i['go']]

True

In [ ]:
#checking for special characters in German dictionary
for i in sorted (ger_w2i.keys())[:20] :  
     print(i, end = " ") 

! $ - . <EOS> <PAD> <SOS> ? ab abbiegen abend abendbrot abendessen abenteuer aber abgefahren abgekommen abgelaufen abgelehnt abgelenkt 

In [ ]:
def sent_to_ind(sentence,lang):
  '''
  Tokenizes a string and
  converts it to an np array of 
  indices and pads the 
  array according to max sentence length
  '''
  ind_list=[]
  if lang=='eng':
    tokens=eng.tokenize(sentence)
    for token in tokens:
      ind_list.append(eng_w2i[token])
    while len(ind_list)<max(ger.maxlen,eng.maxlen):
      ind_list.append(0)
  else:
    tokens=ger.tokenize(sentence)
    for token in tokens:
      ind_list.append(ger_w2i[token])
    while len(ind_list)<max(ger.maxlen,eng.maxlen):
      ind_list.append(0)
    
  return np.array(ind_list)



In [ ]:
#Checking correctness of sentence to index conversion
test_lis=sent_to_ind('<SOS> tom wird bald hier sein . was heißt bald ? <EOS>','ger')

print(len(test_lis))
print(test_lis)

13
[   1 4003 4686  309 1988 3604    6 4568 1946  309    7    2    0]


In [ ]:
test_lis=sent_to_ind("<SOS> tom will be here soon . how soon ? <EOS>",'eng')
print(len(test_lis))
print(test_lis)

13
[   1 3269 3557  262 1477 2927    6 1551 2927    7    2    0    0]


In [ ]:
#splitting the data into training,testing and validation sets

train_x,test_x,train_y,test_y=train_test_split(df['eng'],df['ger'],test_size=0.1,random_state=42)
train_x,val_x,train_y,val_y=train_test_split(train_x,train_y,test_size=0.23,random_state=42)


In [ ]:
def sent_to_np(series,lang,translate_mode):
  '''
  Converts a dataframe column to 
  a unsqueezed np array of indexes
  with padding for feeding into NN
  '''
  ret_list=[]
  if translate_mode==False :
    if lang=='eng':
      for index,val in series.items():
        ret_list.append(sent_to_ind(val,'eng'))
    else:
      for index,val in series.items():
        ret_list.append(sent_to_ind(val,'ger'))
    
    ret_list=np.array(ret_list)
    return np.expand_dims(ret_list,axis=2)
  else:
    ans=sent_to_ind(series,'eng')
    ans=np.expand_dims(ans,axis=0)
    ans=np.expand_dims(ans,axis=2)
    return ans


In [ ]:
train_x=sent_to_np(train_x,'eng',False)
train_y=sent_to_np(train_y,'ger',False)
test_x=sent_to_np(test_x,'eng',False)
test_y=sent_to_np(test_y,'ger',False)
val_x=sent_to_np(val_x,'eng',False)
val_y=sent_to_np(val_y,'ger',False)


In [ ]:
#loading training,testing and validation data

# train_x=np.load("drive/MyDrive/data/training/train_x.npy")
# train_y=np.load("drive/MyDrive/data/training/train_y.npy")

# test_x=np.load("drive/MyDrive/data/testing/test_x.npy")
# test_y=np.load("drive/MyDrive/data/testing/test_y.npy")

# val_x=np.load("drive/MyDrive/data/validation/val_x.npy")
# val_y=np.load("drive/MyDrive/data/validation/val_y.npy")



In [ ]:
train_y=to_categorical(train_y,num_classes=len(ger.vocab_list))
test_y=to_categorical(test_y,num_classes=len(ger.vocab_list))
val_y=to_categorical(val_y,num_classes=len(ger.vocab_list))
print(train_y.shape)
print(test_y.shape)
print(val_y.shape)


(10291, 13, 4892)
(1485, 13, 4892)
(3074, 13, 4892)


In [ ]:
# [batch,timesteps,feature]
print(train_x.shape)
print(train_y.shape)

(10291, 13, 1)
(10291, 13, 4892)


In [ ]:
print(test_x.shape)
print(test_y.shape)

(1485, 13, 1)
(1485, 13, 4892)


In [ ]:
print(val_x.shape)
print(val_y.shape)

(3074, 13, 1)
(3074, 13, 4892)


In [ ]:
def translate(sentence,mod,embedded):
  '''
  Function for translating given English sentence
  to German using model predictions
  '''
  ans=""
  preproc_sent=process_text(sentence,True)
  #print(preproc_sent)
  model_inp=sent_to_np(preproc_sent,'eng',True)
  if (embedded):
    model_inp=np.squeeze(model_inp,axis=2)
  #print(model_inp)
  pred=mod.predict(model_inp)
  #print(pred)
  for i in pred[0]:
    ind=np.argmax(i)
    #print(ind)
    #print("MAX:",i[ind])
    #print(i[24])
    if(ger_i2w[ind]=='<SOS>' or ger_i2w[ind]=='<EOS>' or ger_i2w[ind]=='<PAD>'):
        continue
    ans+=ger_i2w[ind]
    ans+=" "
  return ans

In [ ]:
#training starts here ----------------------------------------------------------

In [ ]:

#Removing singleton axis  3rd axis
#for embedding layer
t_x=np.squeeze(train_x,axis=2)
v_x=np.squeeze(val_x,axis=2)
te_x=np.squeeze(test_x,axis=2)
print(t_x.shape)
print(v_x.shape)
print(te_x.shape)

(10291, 13)
(3074, 13)
(1485, 13)


In [ ]:
def f1_score(y_true,y_pred):
  p=tf.keras.metrics.Precision()
  r=tf.keras.metrics.Recall()
  p.update_state(y_true,y_pred)
  r.update_state(y_true,y_pred)
  return (2*p.result()*r.result())/(p.result()+r.result());
def leaky_relu(x):
    return tf.nn.leaky_relu(x, alpha=0.03)


def base_LSTM_model_HP_TUNING(hp):
  '''
  Simple LSTM model for hyperparameter tuning
  '''
  m1_lstm_units=hp.Int('units',min_value=32,max_value=128,step=32)
  m1_lr=hp.Choice('learning_rate', values = [0.001,0.003,0.005,0.008,0.01,0.05]) 
  lstm=LSTM(m1_lstm_units,return_sequences=True,activation='tanh')  #LSTM layer with output being hiddent state at time t
  layer_at_t=TimeDistributed(Dense(len(ger.vocab_list),activation='softmax')) #Dense layer acting on hidden output at each step to generate predictions
  model=tf.keras.Sequential()
  model.add(InputLayer(train_x.shape[1:]))
  model.add(lstm)
  model.add(layer_at_t)

  model.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(m1_lr),metrics=['accuracy','MeanSquaredError',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
  return model


In [ ]:



def embedding_LSTM_HP_TUNING(hp):
  '''
  LSTM model with embedding layer for hyperparameter tuning
  '''
  m2_lstm_units=hp.Int('units',min_value=32,max_value=128,step=32)
  m2_lr=hp.Choice('learning_rate', values = [0.001,0.003,0.005,0.008,0.01,0.05,0.08])
  embedding_col=hp.Int('output_dim',min_value=32,max_value=160,step=32)
  lstm=LSTM(m2_lstm_units,return_sequences=True,activation='tanh')
  print(t_x.shape[1])
  embed=Embedding(len(ger.vocab_list),embedding_col,input_length=t_x.shape[1])
  layer_at_t=TimeDistributed(Dense(len(ger.vocab_list),activation="softmax"))
  model=tf.keras.Sequential()
  model.add(embed)
  model.add(lstm)
  model.add(layer_at_t)

  model.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(m2_lr),metrics=['accuracy','MeanSquaredError',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
  return model
  

In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)



In [ ]:
#Hyperparameter tuning starts here
#Hyperparameter tuning of base model
# tuner = kt.BayesianOptimization(base_LSTM_model_HP_TUNING,
#                      objective = 'val_accuracy', 
#                      num_initial_points=50,
#                      max_trials=15,
#                      directory='./',
#                      project_name='base_LSTM')


In [ ]:
tuner.search(train_x,train_y, epochs = 25, validation_data = (val_x, val_y), callbacks = [ClearTrainingOutput()])

Trial 16 Complete [00h 02m 53s]
val_accuracy: 0.7552497386932373

Best val_accuracy So Far: 0.765637218952179
Total elapsed time: 00h 47m 45s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_hps1 = tuner.get_best_hyperparameters(num_trials = 1)[0]

In [ ]:
best_hps1.get('units')

128

In [ ]:
best_hps1.get('learning_rate')

0.001

In [ ]:
tuner2 = kt.BayesianOptimization(embedding_LSTM_HP_TUNING,
                     objective = 'val_accuracy', 
                     num_initial_points=50,
                     max_trials=30,
                     directory='./',
                     project_name='embedding')


31


In [ ]:
tuner2.search(t_x,train_y, epochs = 30, validation_data = (v_x, val_y), callbacks = [ClearTrainingOutput()])


Trial 30 Complete [00h 07m 08s]
val_accuracy: 0.8056359887123108

Best val_accuracy So Far: 0.8067874908447266
Total elapsed time: 03h 46m 53s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_hps2 = tuner2.get_best_hyperparameters(num_trials = 1)[0]

In [ ]:
best_hps2.get('units')

128

In [ ]:
best_hps2.get('learning_rate')

0.01

In [ ]:
best_hps2.get('output_dim')

96

In [ ]:
#Making base model using best hyperparameters
def base_LSTM_model(m1_lstm_units,m1_lr):
  '''
  Simple LSTM model
  '''
  lstm=LSTM(m1_lstm_units,return_sequences=True,activation='tanh')  #LSTM layer with output being hiddent state at time t
  layer_at_t=TimeDistributed(Dense(len(ger.vocab_list),activation='softmax')) #Dense layer acting on hidden output at each step to generate predictions
  model=tf.keras.Sequential()
  model.add(InputLayer(train_x.shape[1:]))
  model.add(lstm)
  model.add(layer_at_t)

  model.compile(loss=categorical_crossentropy,optimizer=Adam(m1_lr),metrics=['accuracy','MeanSquaredError',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
  return model

In [ ]:
#Making embedding model using hyperparameters
def embedding_LSTM(m2_lstm_units,m2_lr,embedding_col):
  '''
  LSTM model with embedding layer
  '''
  lstm=LSTM(m2_lstm_units,return_sequences=True,activation='tanh')
  print(t_x.shape[1])
  embed=Embedding(len(ger.vocab_list),embedding_col,input_length=t_x.shape[1])
  layer_at_t=TimeDistributed(Dense(len(ger.vocab_list),activation="softmax"))
  model=tf.keras.Sequential()
  model.add(embed)
  model.add(lstm)
  model.add(layer_at_t)

  model.compile(loss=categorical_crossentropy,optimizer=Adam(m2_lr),metrics=['accuracy','MeanSquaredError',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
  return model

In [ ]:
base_model=base_LSTM_model(128,0.001)
base_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 13, 128)           66560     
_________________________________________________________________
time_distributed_6 (TimeDist (None, 13, 4892)          631068    
Total params: 697,628
Trainable params: 697,628
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#training the base model

base_model.fit(train_x,train_y,m1_batch_size,m1_epochs,validation_data=(val_x,val_y))
#base_model.save("Less_trained_base.h5")

Train on 10291 samples, validate on 3074 samples
Epoch 1/25
10291/10291 [==============================] - 5s 527us/sample - loss: 4.4040 - acc: 0.4942 - mean_squared_error: 1.4624e-04 - precision_7: 0.8602 - recall_6: 0.2758 - val_loss: 2.7209 - val_acc: 0.5539 - val_mean_squared_error: 1.0406e-04 - val_precision_7: 0.9218 - val_recall_6: 0.4873
Epoch 2/25
10291/10291 [==============================] - 5s 449us/sample - loss: 2.6246 - acc: 0.6319 - mean_squared_error: 1.0174e-04 - precision_7: 0.9256 - recall_6: 0.4815 - val_loss: 2.5904 - val_acc: 0.6359 - val_mean_squared_error: 1.0024e-04 - val_precision_7: 0.9224 - val_recall_6: 0.4870
Epoch 3/25
10291/10291 [==============================] - 5s 447us/sample - loss: 2.4948 - acc: 0.6522 - mean_squared_error: 1.0012e-04 - precision_7: 0.9298 - recall_6: 0.4784 - val_loss: 2.4443 - val_acc: 0.6540 - val_mean_squared_error: 9.9470e-05 - val_precision_7: 0.9225 - val_recall_6: 0.4871
Epoch 4/25
10291/10291 [===========================

In [ ]:
# base_model=tf.keras.models.load_model("drive/MyDrive/model/Less_trained_base.h5")
# base_model.summary()

In [ ]:
#Evaluating LSTM model on unseen data

l,acc,mse,p,r=base_model.evaluate(test_x,test_y)
print("Base model loss for testing set:{}".format(l))
print("Base model accuracy for testing set:{}".format(acc))
print("Base model precision for testing set:{}".format(p))
print("Base model recall for testing set:{}".format(r))
print("Base model f1_score for testing set:{}".format((2*p*r)/(p+r)))

1485/1485 [==============================] - 1s 400us/sample - loss: 1.7264 - acc: 0.7080 - mean_squared_error: 7.6027e-05 - precision_7: 0.8947 - recall_6: 0.6224
Base model loss for testing set:1.7263565836530743
Base model accuracy for testing set:0.7080031037330627
Base model precision for testing set:0.8947054743766785
Base model recall for testing set:0.6223776340484619
Base model f1_score for testing set:0.73409907650077


In [ ]:
#Sample translation by base LSTM model
translate("work here",base_model,False)

'tom ist ! ! '

In [ ]:
embeded_model=embedding_LSTM(96,0.01,128)
embeded_model.summary()

13
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 13, 64)            313088    
_________________________________________________________________
lstm_9 (LSTM)                (None, 13, 96)            61824     
_________________________________________________________________
time_distributed_9 (TimeDist (None, 13, 4892)          474524    
Total params: 849,436
Trainable params: 849,436
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#training embedding model
embeded_model.fit(t_x,train_y,batch_size=128,epochs=15,validation_data=(v_x,val_y))

Train on 10291 samples, validate on 3074 samples
Epoch 1/15
10291/10291 [==============================] - 6s 582us/sample - loss: 3.2519 - acc: 0.5582 - mean_squared_error: 1.2217e-04 - precision_10: 0.7860 - recall_9: 0.4688 - val_loss: 2.2596 - val_acc: 0.6592 - val_mean_squared_error: 8.3884e-05 - val_precision_10: 0.9566 - val_recall_9: 0.5594
Epoch 2/15
10291/10291 [==============================] - 5s 472us/sample - loss: 2.0892 - acc: 0.6905 - mean_squared_error: 7.9235e-05 - precision_10: 0.9608 - recall_9: 0.5665 - val_loss: 1.9971 - val_acc: 0.7094 - val_mean_squared_error: 7.5222e-05 - val_precision_10: 0.9412 - val_recall_9: 0.5969
Epoch 3/15
10291/10291 [==============================] - 5s 477us/sample - loss: 1.6992 - acc: 0.7253 - mean_squared_error: 6.9341e-05 - precision_10: 0.9387 - recall_9: 0.6597 - val_loss: 1.6192 - val_acc: 0.7363 - val_mean_squared_error: 6.7545e-05 - val_precision_10: 0.9228 - val_recall_9: 0.6767
Epoch 4/15
10291/10291 [=====================

In [ ]:
#embeded_model=tf.keras.models.load_model("drive/MyDrive/embedded_15000.h5")
l,acc,mse,p,r=embeded_model.evaluate(te_x,test_y)
print("Embedded model loss for testing set:{}".format(l))
print("Embedded model accuracy for testing set:{}".format(acc))
print("Embedded model MSE for testing set:{}".format(mse))
print("Embedded model precision for testing set:{}".format(p))
print("Embedded model recall for testing set:{}".format(r))
print("Embedded model f1_score for testing set:{}".format((2*p*r)/(p+r)))


1485/1485 [==============================] - 1s 427us/sample - loss: 1.1761 - acc: 0.8134 - mean_squared_error: 5.4238e-05 - precision_10: 0.8990 - recall_9: 0.7738
Embedded model loss for testing set:1.17609374153895
Embedded model accuracy for testing set:0.8134162425994873
Embedded model MSE for testing set:5.4237843869486824e-05
Embedded model precision for testing set:0.8990190625190735
Embedded model recall for testing set:0.7738409638404846
Embedded model f1_score for testing set:0.8317465500860717


In [ ]:
translate("i am so small ",embeded_model,True)

'ich bin so klein . '